Jusqu'à présent, je me suis concentré sur le fichier train.csv mais il y a aussi authorData qui n'a juste pas l'embedding.

J'avais vu que l'embedding n'était pas méga importante mais je vais chercher à la garder. Pour cela, je vais chercher à estimer lesparamètres de l'embedding.

(Notations : D1 = train.csv; D2 = authorData.csv; N = colonnes non-embedding(=explicites parce que je les connais presque qu'elles sont presques lisibles par un humain); M = colonnes embeddées.)

idée : utiliser train pour apprendre l'embedding sur N colonnes pour en prédire m<=M.
M=1024 donc prédire les 1024 est violent. On se propose de prédire les m termes de la pca

Etapes (d'entrainement et calibration du modèle)
* PCA(m) sur M colonnes de train
* diff train, test et X, Y 
* prédiction(m) à partir de N de X_train
  * modèle
  * métrique

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity


from notebook_helper import split_url, explicit_part_preparation, pca_transform

In [2]:
df = pd.read_csv("./data/train.csv", sep=",")

### a. PCA

In [3]:
m = 20
df_reduced = pca_transform(df, n_components=m)

In [4]:
df_reduced.head(3)

,engagement,timestamp,language,feature1,feature2,followers,author,word_count,shared_url_count,shared_url_domain,...,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
0,15,1655784230254,en,73,0,696,bd20432d80dfe4825a7a106312bda52e,18,0,NaN,...,-0.009545,-0.122285,0.033629,0.023458,-0.047361,-0.011103,-0.023807,0.058836,-0.035002,-0.027253
1,91007,1655599613254,en,100,5,17526,71e2ecc6cc1d6515a6b122fab4c63bfc,15,0,NaN,...,-0.099002,0.003194,0.020019,0.005192,-0.063563,-0.017077,-0.001559,0.083150,-0.049121,0.033752
2,3,1655788872254,en,0,-5,239,2a9109f38d90a2a96e284a07c1a57e9b,8,0,NaN,...,-0.026192,-0.009343,0.065408,0.000540,-0.002416,-0.018255,-0.018738,-0.014282,-0.048993,-0.005838


### b. entraînement

In [5]:
df_prepared = explicit_part_preparation(df_reduced)

In [6]:
df_prepared.head(3)

,engagement,timestamp,feature2,followers,word_count,is_reply,contains_video,contains_image,V1,V2,...,shared_url_extension_net,shared_url_extension_news,shared_url_extension_org,shared_url_extension_other,shared_url_extension_tv,is_shared_url,month_post,day_of_week_post,hour_post,feature1_is_max
0,1.176091,6.727053,0,2.842609,1.278754,False,False,True,0.040117,0.039279,...,False,False,False,True,False,False,6,1,4,False
1,4.959075,8.278642,5,4.243683,1.204120,False,False,False,-0.154545,0.130231,...,False,False,False,True,False,False,6,6,0,True
2,0.477121,5.840107,-5,2.378398,0.954243,True,False,False,0.319861,-0.050030,...,False,False,False,True,False,False,6,1,5,False


In [7]:
V_cols = [col for col in df_prepared.columns if "V" in col]

# Separation of the target
X = df_prepared.drop(columns=V_cols)
y = df_prepared[V_cols]

# Test-train split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [9]:
y_pred_dict = {}

for v_col in V_cols:
    print(v_col)

    # Model training
    model = RandomForestRegressor()
    model.fit(X_train, y_train[v_col])

    # Predict
    y_pred_dict[v_col] = model.predict(X_test)

y_pred = pd.DataFrame(y_pred_dict).T


V1
V2
V3
V4
V5
V6
V7
V8
V9
V10
V11
V12
V13
V14
V15
V16
V17
V18
V19
V20


In [10]:
for v_col in V_cols:
    mse = mean_squared_error(y_test[v_col], y_pred_dict[v_col])
    print(f"{v_col} mse = {mse}")

V1 mse = 0.013525531153525581
V2 mse = 0.004656586337104673
V3 mse = 0.007978101352030585
V4 mse = 0.0028228398588694636
V5 mse = 0.0036789499024297246
V6 mse = 0.0025490375182393374
V7 mse = 0.0023351353081270416
V8 mse = 0.001753694258768791
V9 mse = 0.0017808853426720787
V10 mse = 0.0017540391789052057
V11 mse = 0.0018450527960957328
V12 mse = 0.0013513661639202555
V13 mse = 0.0014689032705727564
V14 mse = 0.0014400164550749757
V15 mse = 0.0013585717083815958
V16 mse = 0.0014080944823454539
V17 mse = 0.0013412243024324528
V18 mse = 0.0012568870654133984
V19 mse = 0.0014270579198440454
V20 mse = 0.0012472665949648234


In [27]:
for v_col in V_cols:
    sim  = cosine_similarity(
        np.array(y_test[v_col].to_list()).reshape(1, -1),
        y_pred_dict[v_col].reshape(1, -1),
    ).diagonal()
    print(f"{v_col} sim = {sim.mean()}")

V1 sim = 0.634263576624334
V2 sim = 0.8065995881237711
V3 sim = 0.4635025495785349
V4 sim = 0.5888221188097175
V5 sim = 0.3390623640206514
V6 sim = 0.38389123874527625
V7 sim = 0.3040292161277471
V8 sim = 0.43820734629609204
V9 sim = 0.2393079673774707
V10 sim = 0.2819773142695649
V11 sim = 0.1542468947242968
V12 sim = 0.3327769944208841
V13 sim = 0.18619592509603222
V14 sim = 0.1980749025102401
V15 sim = 0.1641799010677395
V16 sim = 0.16349072330763767
V17 sim = 0.15735126610477987
V18 sim = 0.13628041062359142
V19 sim = -0.021849928519963587
V20 sim = 0.084125426124727


Les premières composantes (V1–V4) ont une cos-sim correcte à bonne (0.63–0.80).

À partir de V5, ça chute vite vers 0.3, 0.2… voire négatif pour V19...




* Les premières composantes PCA concentrent la variance structurelle (elles sont bien reconstruites).
* Les dernières composantes sont plus bruitées ou faiblement corrélées à tes features X_N — donc impossibles à prédire proprement.

Ce pattern est normal en PCA : les composantes au-delà de la 10ᵉ sont souvent du “bruit résiduel”.